In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import torch
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from Scripts.DataManager.GraphConstructor.CoOccurrenceGraphConstructor import CoOccurrenceGraphConstructor

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')
from Scripts.Configs.ConfigClass import Config
config = Config(r'C:\Users\fardin\Projects\ColorIntelligence\Scripts\Configs\Config.json')
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx

In [2]:
train_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\train_sm.csv')
test_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\test_sm.csv')

In [3]:
train_df.columns = ['Polarity', 'Title', 'Review']
test_df.columns = ['Polarity', 'Title', 'Review']

In [4]:
train_df = train_df[['Polarity', 'Review']]
test_df = test_df[['Polarity', 'Review']]

In [5]:
my_dict = {1: 'abc', 3: 'aaa'}
if 3 in my_dict:
    print(my_dict[1])

abc


In [6]:
len(train_df['Review'])

50000

In [7]:
#train_df['Review'][:10]

In [8]:
config = Config(r'C:\Users\fardin\Projects\ColorIntelligence\Scripts\Configs\Config.json')

In [ ]:
# graph_const = CoOccurrenceGraphConstructor(None, 'AmazonReview', config, lazy_construction=False, naming_prepend='graph', load_preprocessed_data=True)
graph_const = CoOccurrenceGraphConstructor(train_df['Review'], 'AmazonReview', config, lazy_construction=False, naming_prepend='graph', load_preprocessed_data=False)
# graph = graph_const.to_graph(train_df['Review'][0])

i: 0
i: 100
i: 200
i: 300
i: 400
i: 500
i: 600
i: 700
i: 800
i: 900
i: 1000
i: 1100
i: 1200
i: 1300
i: 1400
i: 1500
i: 1600
i: 1700
i: 1800
i: 1900
i: 2000
i: 2100
i: 2200
i: 2300
i: 2400
i: 2500
i: 2600
i: 2700
i: 2800
i: 2900
i: 3000
i: 3100
i: 3200
i: 3300
i: 3400
i: 3500
i: 3600
i: 3700
i: 3800
i: 3900
i: 4000
i: 4100
i: 4200
i: 4300
i: 4400
i: 4500
i: 4600
i: 4700
i: 4800
i: 4900
i: 5000
i: 5100
i: 5200
i: 5300


In [ ]:
print(len(graph_const.graphs))

In [ ]:
type(graph_const.graphs[0])

In [22]:
import os

In [24]:
os.path.join(graph_const.save_path, graph_const.var.graphs_name[0])

'C:\\Users\\fardin\\Projects\\ColorIntelligence\\data\\GraphData\\AmazonReview\\graph_0.graph'

In [13]:
g = to_networkx(graph_const.graphs[0], to_undirected=True)
layout = nx.spring_layout(g)
nx.draw(g, pos=layout)
unique_words_dict = {i: graph_const.unique_words[i] for i in range(len(graph_const.unique_words))}
nx.draw_networkx_labels(g, pos=layout, labels=unique_words_dict)

AttributeError: 'NoneType' object has no attribute 'num_nodes'

In [6]:
doc_graph.edge_index

NameError: name 'doc_graph' is not defined

In [13]:
len(graph_const.unique_words)

104

In [8]:
[(x,y) for x in range(3) for y in range(3) if x!=y]

[(0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1)]